# HuggingFace LLM과 공개 데이터셋을 활용한 RLHF

## **1. Imports**

In [ ]:
## ============= Basic Modules =============
import pandas as pd
import numpy as np
import random
import torch
from dataclasses import dataclass, field, fields    ## For TrlParser

## ================ Utility ================
import os
import gc
import argparse
import wandb        ## External Training Log Analysis Tool

## ========== Hugging Face Library ==========
from huggingface_hub import login   ## Input Token

import datasets
from datasets import load_dataset

from trl import (
    SFTTrainer, SFTConfig,  ## SFT
    DPOConfig, DPOTrainer,  ## DPO
    TrlParser               ## YAML Parameter Parsing
)

from transformers import (
    AutoModelForCausalLM, AutoTokenizer,    ## Vanilla Model Loading
    HfArgumentParser, TrainingArguments,    ## Tuning Parameter Setting
    BitsAndBytesConfig,                     ## Quantization
    set_seed                                ## Seeding (not recommended)
)

from peft import (
    LoraConfig,                             ## For Low-Rank Adaption
    PeftConfig, AutoPeftModelForCausalLM, PeftModel
)

## **2. UltraFeedback Dataset**

* 고품질 AI 피드백 데이터셋. PPO에 활용되는 데이터셋을 이진 선호도 데이터셋으로 변환한 형태(RLAIF)
* 특정 태스크에 목적이 있는 게 아닌, 다양한 주제에서의 채팅 기능 향상을 위한 데이터셋

`-` 데이터셋 로드

In [ ]:
## 원시 데이터 로드
ds = load_dataset("argilla/ultrafeedback-binarized-preferences-cleaned")

`-` SFT / DPO 데이터셋 분리

* 본래는 SFT를 위한 데이터셋과 DPO를 위한 데이터셋이 따로 준비되어 있어야 하지만, DPO 데이터셋만 존재하기 때문에 실습을 위해 둘을 분리하여 훈련을 진행합니다.
* 실제 추천 프레임워크

    1. 원하는 태스크의 라벨링된 데이터셋으로 SFT 수행
    2. SFT를 완수한 모델을 이용하여 한 질문에 대하여 두 개 이상의 답변 생성
    3. 생성된 답변을 조합하여 Chosen / Rejected 쌍으로 구성된 선호도 데이터셋 제작(HF or AIF)
    > 해당 과정에서 가장 좋은 답변만을 Chosen으로 설정할 수 있음
    >
    > n개 답변을 생성했다면, 그중 한 개의 답변을 최선으로 설정한 뒤, 이를 Chosen으로 설정. 총 n-1개의 선호도 쌍을 생성 가능
    >
    > 일반적으로 순위를 매긴 뒤, 상위 몇 개의 답변만 Chosen으로 설정한 다음 조합하여 선호도 쌍을 생성
    4. 선호도 데이터셋을 사용하여 SFT를 수행한 모델에 DPO를 이어서 적용

In [ ]:
os.makedirs("data", exist_ok = True)    ## 디렉토리 생성

ds_split = ds["train"].train_test_split(test_size = 0.5, seed = 42) ## SFT/DPO 데이터 분할

`-` 데이터셋 전처리

In [ ]:
## For SFT
sft_ds = ds_split["train"]
sft_ds = sft_ds.rename_column("chosen", "messages").remove_columns([col for col in sft_ds.column_names if col != "chosen"]).train_test_split(test_size = 0.1, seed = 42)
sft_ds["train"].to_json("./data/sft_train_dataset.json", orient = "records")
sft_ds["test"].to_json("./data/sft_test_dataset.json", orient = "records")

## For DPO: Implicit Prompt -> Explicit Prompt (Recommanded)
dpo_ds = ds_split["test"].map(
    lambda sample: {
        "prompt": [{"role": "user", "content": sample["prompt"]}],
        "chosen": [content for content in sample["chosen"] if content["role"] == "assistant"],
        "rejected": [content for content in sample["rejected"] if content["role"] == "assistant"]
    }
)

dpo_ds = dpo_ds.remove_columns([col for col in dpo_ds.column_names if col not in ["prompt", "chosen", "rejected"]]).train_test_split(test_size = 0.1, seed = 42)
dpo_ds["train"].to_json("./data/dpo_train_dataset.json", orient = "records")
dpo_ds["test"].to_json("./data/dpo_test_dataset.json", orient = "records")

## **3. Model Loading**

* 허깅페이스에 로그인하여 권한을 획득하고, [Llama-3.1-8B-Instruct](https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct) 모델을 로드

`-` 허깅페이스에 로그인

* Access Token을 발급받은 뒤, 모델 사용 권한을 요청하면 잠시 뒤 모델에 접근할 수 있습니다.
* 허깅페이스 프로필 아이콘 클릭 > Access Tokens에서 발급 가능
* [Llama-3.1-8B-Instruct](https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct) 모델 접근 권한 획득

In [ ]:
login(token = "hf_...")

> 해당 방식은 시연을 위한 방법이고, `login()`으로 작성하여 `token` 파라미터를 사용하지 않은 상태에서 터미널에서 토큰을 입력해 로그인하는 것이 안전합니다.
>
> 일반적으로는 터미널에서 `hf auth login` 명령어를 통해 미리 로그인해주는 것이 편합니다. 이러면 해당 로그인 코드는 필요하지 않습니다.

`-` 모델 및 토크나이저 로드

* 거의 대부분의 모델은 두 개의 객체를 함께 로드합니다.
* 토크나이저(Tokenizer): 텍스트를 단어(실제론 이보다 작은 단위)로 쪼갠 다음(Tokenize), Token ID로 매핑
* 모델(Model): 임베딩 후 생성

    > 양자화 설정을 여기서 수행

In [ ]:
## 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-3.1-8B-Instruct",
    use_fast = True,            ## Rust로 구현된 Fast Tokenizer 사용 (Qwen, RoPE, ChatGLM 등의 특이한 구조에서는 호환 안됨)
    trust_remote_code = True    ## 모델 코드 전체 다운로드 후 사용
)

## 양자화 설정: 모델의 가중치를 로드할 때 양자화하여 들여옵니다.
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,                    ## 4비트로 양자화
    bnb_4bit_use_double_quant = True,       ## 추가 양자화(스케일 파라미터 양자화) 활성화. 메모리 절약
    bnb_4bit_quant_type = "nf4",            ## 양자화 데이터 타입 지정: 4비트 기반 모델 훈련 시 사용
    bnb_4bit_compute_dtype = torch.bfloat16 ## 4비트로 로드하지만, attention 연산 시 해당 포맷으로 역양자화하여 처리 (라마 기본 자료형, 거의 대부분의 최신 LLM은 해당 포맷 사용)
)

## 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.1-8B-Instruct",         ## 사용 모델명
    device_map = "cuda:0",                      ## GPU 사용
    use_cache = False,                          ## KV 캐시 미사용(VRAM), 추론 속도 저하. gradienc_checkpointing과 동시 사용 불가
    attn_implementation = "flash_attention_2",  ## flash_attention 연산 사용. sdpa가 더 빠르고 효율적일 수도 있음.
    dtype = torch.bfloat16,                     ## 초기 가중치 로드 데이터 타입. Llama-3.1-8B의 자료형으로 설정
    quantization_config = bnb_config            ## 양자화 설정 적용
)

`-` 모델 및 토크나이저 설정

* 토크나이저를 일부 커스터마이징합니다. Chat Template 설정이 주요 요소입니다.
* Chat Template는 기본적인 템플릿이 따로 존재하지만, SFT에서 `assistant_only_loss`로 학습을 위해서는 추가적인 양식이 요구됩니다.

    > `prompt-completion` 포맷의 데이터셋은 `completion_only_loss`를 활성화하는 것만으로도 간단하게 라벨 부분만으로 손실을 계산할 수 있습니다.
    >
    > 일부 모델의 토크나이저에는 `generation` 부분이 템플릿에 포함되어 있어 그대로 사용해도 될 수 있습니다.

In [ ]:
tokenizer.pad_token = tokenizer.eos_token       ## 패딩할 토큰 설정 (padding_free 설정 시 큰 의미는 없음)
tokenizer.padding_side = "left"                 ## 디코더이므로 왼쪽을 패딩 (마지막 토큰을 보고 생성)

## 데이터셋에 적합한 chat template 적용: {% generation %} 부분을 추가하여 assistant_only_loss 진행
## 모든 텍스트로 손실을 계산하고자 한다면 tokenizer에 기본으로 할당된 chat template로 충분
## jinja2 template engine 구문. 파이썬 문법과 거의 동일
LLAMA_3_CHAT_TEMPLATE = (
    "{{ bos_token }}"
    "{% for message in messages %}"
        "{% if message['role'] == 'system' %}"
            "{{ '<|start_header_id|>system<|end_header_id|>\n\n' + message['content'] + eos_token }}"
        "{% elif message['role'] == 'user' %}"
            "{{ '<|start_header_id|>user<|end_header_id|>\n\n' + message['content'] +  eos_token }}"
        "{% elif message['role'] == 'assistant' %}"
            "{{ '<|start_header_id|>assistant<|end_header_id|>\n\n'}}"
            "{% generation %}"
            "{{ message['content'] +  eos_token }}"
            "{% endgeneration %}"
        "{% endif %}"
    "{% endfor %}"
    "{%- if add_generation_prompt %}"
    "{{- '<|start_header_id|>assistant<|end_header_id|>\n\n' }}"
    "{%- endif %}"
)

## 새로운 Chat Template 적용
tokenizer.chat_template = LLAMA_3_CHAT_TEMPLATE

> 핵심은 `'assistant'`부분에 `{% generation %} ~ {% endgeneration %}`을 추가하는 것입니다. 이러면 해당 텍스트는 마스킹에서 제외됩니다.
>
> ```Python
> "{{ '<|start_header_id|>assistant<|end_header_id|>\n\n'}}"
> "{% generation %}"
> "{{ message['content'] +  eos_token }}"
> "{% endgeneration %}"
> ```

## **4. Supervised Fine Tuning (SFT)**

* 기본 모델을 원하는 태스크에 특화되도록 1차적인 파인튜닝을 시행합니다. 단일 GPU 환경을 가장합니다. 분산 GPU 환경에서의 학습은 [FSDP](https://huggingface.co/docs/transformers/ko/fsdp)를 참고하세요.
* `YAML` 파일로 하이퍼파라미터를 쉽게 관리할 수 있습니다.

    > 단순 `argparse` 모듈로 파라미터 설정을 할 수도 있지만, 이 경우 `accelerate` 라이브러리를 통한 분산 GPU 환경에서의 학습에 호환되지 않으므로 확장성을 위해 이와 같은 방식을 택했습니다.

`(선택사항)`

* 학습 과정에서 계산된 손실, 평균 토큰 정확도 등의 로그를 wandb를 통해 간단히 시각화/분석할 수 있습니다.
* wandb 플랫폼에 로그인이 필요합니다. https://wandb.ai/site

`-` 사전 설정

In [ ]:
## Zombie Process 발생 방지
os.environ["WANDB_MODE"] = "offline"    ## 수동 업데이트: wandb sync --include-offline ./wandb/latest-run
wandb.init(project = "Test Project")    ## wandb에서 만든 프로젝트 이름 명시

## 파이토치 백엔드에서 fp32대신 tf32 자료형을 사용하여 처리량 개선 (약간의 정확도 감소)
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

`-` 유틸리티 구성

* `@dataclass` 데코레이터를 활용하여 `TrlParser`에 들어갈 파라미터를 지정해줍니다.
    
    > 현재 SFT의 설정을 조정하는 `SFTConfig` 객체는 `TrlParser`에 완벽히 호환되나, 그 외의 config 객체들은 호환되지 않습니다.
    >
    > 따라서 `ScriptArguments`와 `LoraArguments`라는 클래스를 만들어 튜닝 파라미터를 간단히 수용하고 관리할 수 있게 만들었습니다.

* 총 실행 시간을 계산하는 함수 `timer`와, 재현성을 위한 `seeding` 함수를 정의합니다.

    > 해당 함수들은 선택사항입니다.
    >
    > 시드를 설정할 경우 비결정적 알고리즘을 사용할 수 없게됨에 따라 성능이 일부 감소할 수 있어 사용을 권장하지 않습니다.

In [ ]:
## TrlParser에 들어갈 class들을 커스터마이징: 하이퍼파라미터 저장
@dataclass  ## 데이터 보관 클래스를 간단하게 구축 가능: __init__, __repr__, __eq()__등의 메소드 자동 생성
class ScriptArguments:
    dataset_path: str = field(default = None, metadata = {"help": "dataset directory"})
    model_name: str = field(default = None, metadata = {"help": "사용할 모델 ID"})

@dataclass
class LoraArguments:
    r: int = field(default = 64, metadata = {"help": "update matrix의 rank. 작을수록 많이 압축하여 품질 저하됨, 메모리 많이 할당됨"})
    lora_alpha: int = field(default = 32, metadata = {"help": "∆Weight scaling factor. lora_alpha / r로 스케일링되며, 학습률 조정. 보통 1/2 수준으로 설정"})
    lora_dropout: float = field(default = 0.05, metadata = {"help": "update matrics에서 dropout 적용 확률"})
    bias: str = field(default = "none", metadata = {"help": "update matrix에 bias를 학습할 것인지 선택"})
    task_type: str = field(default = "CAUSAL_LM", metadata = {"help": "학습할 모형이 무엇인지 지정"})
    target_modules: list[str] = field(default = None, metadata = {"help": "학습에 반영할 모듈 설정"})


def timer(func):
    """
    함수 실행 시간 출력
    """
    import time
    import datetime

    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()

        sec = end - start
        worktime = str(datetime.timedelta(seconds=sec)).split(".")[0]
        print(f"Working Time: {worktime}")
        return result

    return wrapper


def seeding(seed):
    """
    시드 설정으로 인해 성능이 저하될 수 있음. dataloader worker에도 시드 설정이 필요할 수 있음
    """
    set_seed(seed)

    torch.manual_seed(seed)                 ## cpu seed
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)        ## gpu seed
        torch.cuda.manual_seed_all(seed)    ## multi-gpu seed

    torch.backends.cudnn.deterministic = True   ## nondeterministic algorithm을 사용하지 않도록 설정
    torch.backends.cudnn.benchmark = False      ## cuDNN의 여러 convolution algorithm들을 실행하고, 벤치마킹하여 가장 빠른 알고리즘 사용: 안함.

    np.random.seed(seed)
    random.seed(seed)

    os.environ["PYTHONHASHSEED"] = str(seed)    ## hash 알고리즘 관련
    os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"   ## oneDNN 옵션 해제. 수치 연산 순서 고정 (성능 저하, 속도 저하)

`-` 메인 학습 코드

In [ ]:
@timer  ## 총 소요 시간 측정 및 출력
def main(script_args, training_args, lora_kwargs):
    ## loading dataset
    train_ds = load_dataset("json", data_files = os.path.join(script_args.dataset_path, "sft_train_dataset.json"), split = "train")
    test_ds = load_dataset("json", data_files = os.path.join(script_args.dataset_path, "sft_test_dataset.json"), split = "train")

    ## 데이터셋 사이즈 출력
    print(f"training dataset size: {train_ds.num_rows}\ntest dataset size: {test_ds.num_rows}")

    if training_args.gradient_checkpointing:
        model.gradient_checkpointing_enable()

    peft_config = LoraConfig(**lora_kwargs)

    trainer = SFTTrainer(
        model = model,
        args = training_args,
        train_dataset = train_ds,
        eval_dataset = test_ds,
        processing_class = tokenizer,
        peft_config = peft_config
    )

    if training_args.assistant_only_loss:
        print("======== Log a first sample from the processed training set ========")
        print(f"masking area: {next(iter(trainer.train_dataset))["assistant_masks"][:100]} ...")

    ## 학습이 중단된 경우 이어서 진행할 수 있도록 설정
    checkpoint = None
    if training_args.resume_from_checkpoint is not None:
        checkpoint = training_args.resume_from_checkpoint

    trainer.train(resume_from_checkpoint = checkpoint)
    trainer.save_model()


if __name__ == "__main__":
    os.makedirs("wandb", exist_ok = True)
    # initial_folders = set(next(os.walk("wandb"))[1])

    ## --config config/SFT_config.yaml 대신 임시로...
    config_path = "./main/config/SFT_config.yaml"
    parser = TrlParser((ScriptArguments, SFTConfig, LoraArguments))         ## 따로 저장된 파라미터 파싱
    script_args, training_args, lora_args = parser.parse_args_and_config(
        args=["--config", config_path]
    )

    ## Lora Config에 유효한 입력값만 받을 수 있도록 커스터마이징. 원래 TrlParser에는 LoraConfig를 넣지 못함
    valid_keys = LoraConfig.__init__.__code__.co_varnames
    lora_kwargs = {
        f.name: getattr(lora_args, f.name)
        for f in fields(lora_args)
        if f.name in valid_keys
    }

    # seeding(training_args.seed)

    main(script_args, training_args, lora_kwargs)

    print("========== 학습 종료 ==========")

    ## ========== wandb 업로드 ==========
    os.system(f"wandb sync --include-offline wandb/latest-run") ## 가장 마지막에 진행한 튜닝 작업의 로그 제출

> 원래는 callback을 위한 클래스 커스터마이징까지 넣으려고 했는데, 내용도 길어지고 필요 없을 수도 있는 부분이라 참고용 링크만 제공해드립니다.

* [Callback (HuggingFace)](https://huggingface.co/docs/transformers/v5.0.0rc1/en/main_classes/callback#transformers.TrainerCallback)
* [TrainerCallback (Github)](https://github.com/huggingface/transformers/blob/v5.0.0rc1/src/transformers/trainer_callback.py#L295)

    `transformers.TrainerCallback` 모듈을 상속하여 클래스 구성한 뒤, 원하는 이름의 메서드를 overriding하면 됩니다.

## **5. (1차) 튜닝 모델 테스트**

* 소수의 샘플만 테스트하고 싶다면, HuggingFace 라이브러리만 사용해도 충분합니다.
* 단순히 validation loss가 최소이거나 마지막 체크포인트를 불러올 수도 있으나, 여러 체크포인트에서 불러온 모델들을 테스트하고 그 중 보았을 때 가장 괜찮은 결과를 생성한 버전을 선택하는 것도 좋습니다.

`-` VRAM 초기화

In [ ]:
del model
del tokenizer
gc.collect()
torch.cuda.empty_cache()

`-` 테스트에 사용될 데이터 선택

In [ ]:
## 무작위 1개 텍스트 추출
test_dataset = load_dataset("json", data_files = os.path.join("", "data/sft_test_dataset.json"), split = "train")
random_idx = random.randint(0, len(test_dataset))

## 출력하여 확인
for k, v in test_dataset[random_idx].items():
    print(f"{k}: {v[0]["content"]}\n\n")

messages = test_dataset[random_idx]["prompt"]
chosen = test_dataset[random_idx]["chosen"]

`-` 원시 모델 결과 확인

In [ ]:
## 모델 로드 및 초기 설정
origin_model_name = "meta-llama/Llama-3.1-8B-Instruct"

origin_model = AutoModelForCausalLM.from_pretrained(origin_model_name, use_cache = False, device_map = "cuda:0", dtype = torch.bfloat16)
origin_tokenizer = AutoTokenizer.from_pretrained(origin_model_name, use_fast = True)
origin_tokenizer.pad_token = origin_tokenizer.eos_token
origin_tokenizer.pad_token_id = origin_tokenizer.eos_token_id
origin_tokenizer.padding_side = "left"

terminators = [origin_tokenizer.eos_token_id]

input_ids = origin_tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,   ## 생성 시에 맨 뒤 generation 시작하라는 프롬프트 삽입
    return_tensors = "pt").to(origin_model.device)

## 생성
outputs = origin_model.generate(
    input_ids,
    max_new_tokens = 512,
    eos_token_id = terminators,
    do_sample = True,
    temperature = 0.7,
    top_p = 0.95
)

response = outputs[0][input_ids.shape[-1]:]
print(f"prompt:\n{messages[0]["content"]}\n")
print(f"chosen:\n{chosen[0]["content"]}\n")
print(f"generate:\n{origin_tokenizer.decode(response, skip_special_tokens = True)}")

`-` 베이스 모델과 어뎁터 로드 및 비교

* 베이스 모델을 4비트로 양자화한 뒤, 어뎁터를 부착

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.bfloat16
)

adapter_name = "../adapter/SFT_Adapter" ## 마지막 체크포인트 로드 (상황에 따라 조정)

## 양자화 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.1-8B-Instruct",
    quantization_config = bnb_config,
    attn_implementation = "eager",
    use_cache = True,
    dtype = torch.bfloat16,
    device_map = "cuda:0"
)

model = PeftModel.from_pretrained(model, adapter_name)  ## 어뎁터 부착
tokenizer = AutoTokenizer.from_pretrained(adapter_name, use_fast = True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"

terminators = [tokenizer.eos_token_id]

## 생성
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,   ## 생성 시에 맨 뒤 generation 시작하라는 프롬프트 삽입
    return_tensors = "pt").to(model.device)

outputs = model.generate(
    input_ids,
    max_new_tokens = 512,       ## prompt 제외 출력 토큰 수. max_length는 prompt 포함
    eos_token_id = terminators,
    do_sample = True,
    temperature = 0.7,
    top_p = 0.95
)

response = outputs[0][input_ids.shape[-1]:]
print(f"prompt:\n{messages[0]["content"]}\n")
print(f"chosen:\n{chosen[0]["content"]}\n")
print(f"generate:\n{tokenizer.decode(response, skip_special_tokens = True)}")

In [ ]:
del model
del tokenizer
gc.collect()
torch.cuda.empty_cache()

## **6. Direct Preference Optimization (DPO)**

* 대부분의 설정은 SFT에서와 동일합니다. 하지만 QLoRA 튜닝 환경에서는 중요한 설정이 하나 있습니다.
* 일반 LoRA는 모델과 어뎁터의 가중치를 단순히 더하는 방식으로 모델을 병합(Merge)할 수 있습니다. 하지만, 양자화를 하게 되면 만약 병합을 진행하였을 시 어뎁터의 훈련된 가중치 정보(16비트)까지 4비트로 뭉개버린 다음, 이것에 어뎁터를 부착하여 DPO 학습을 진행하게 됩니다.

    > 따라서 어뎁터를 새로 부착하지 않고, 현재 선택된 최고의 모델(어뎁터)를 이어서 학습하는 방식을 택합니다.
    >
    > [DPO에서 PEFT 사용 시 가능한 방법(HuggingFace)](https://huggingface.co/docs/trl/dpo_trainer#reference-model-considerations-with-peft)

In [ ]:
@dataclass
class ScriptArguments:
    dataset_path: str = field(default = None, metadata = {"help": "dataset directory"})
    model_name: str = field(default = None, metadata = {"help": "사용할 모델 ID"})
    adapter_name: str = field(default = None, metadata = {"help": "SFT 완료된 어뎁터"})

@timer
def main(script_args, training_args):
    ## loading dataset
    train_ds = load_dataset("json", data_files = os.path.join(script_args.dataset_path, "dpo_train_dataset.json"), split = "train")
    test_ds = load_dataset("json", data_files = os.path.join(script_args.dataset_path, "dpo_test_dataset.json"), split = "train")

    tokenizer = AutoTokenizer.from_pretrained(
        script_args.model_name,
        use_fast = True,
        trust_remote_code = True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"

    LLAMA_3_CHAT_TEMPLATE = (
        "{{ bos_token }}"
        "{% for message in messages %}"
            "{% if message['role'] == 'system' %}"
                "{{ '<|start_header_id|>system<|end_header_id|>\n\n' + message['content'] + eos_token }}"
            "{% elif message['role'] == 'user' %}"
                "{{ '<|start_header_id|>user<|end_header_id|>\n\n' + message['content'] +  eos_token }}"
            "{% elif message['role'] == 'assistant' %}"
                "{{ '<|start_header_id|>assistant<|end_header_id|>\n\n'}}"
                "{{ message['content'] +  eos_token }}"
            "{% endif %}"
        "{% endfor %}"
        "{%- if add_generation_prompt %}"
        "{{- '<|start_header_id|>assistant<|end_header_id|>\n\n' }}"
        "{%- endif %}"
    )

    tokenizer.chat_template = LLAMA_3_CHAT_TEMPLATE

    ## 양자화 설정
    bnb_config = BitsAndBytesConfig(
        load_in_4bit = True,
        bnb_4bit_use_double_quant = True,
        bnb_4bit_quant_type = "nf4",
        bnb_4bit_compute_dtype = torch.bfloat16
    )

    ## 모델 로드 및 설정
    model = AutoModelForCausalLM.from_pretrained(
        script_args.model_name,
        device_map = "cuda:0",
        use_cache = False,
        low_cpu_mem_usage = True,
        attn_implementation = "flash_attention_2",
        trust_remote_code = True,
        quantization_config = bnb_config,
        dtype = torch.bfloat16
    )

    ## 어뎁터 부착
    model = PeftModel.from_pretrained(
        model,
        script_args.adapter_name,
        is_trainable=True,
        adapter_name="policy",
    )

    ## 어뎁터를 두 번 로드하여 하나는 Policy, 하나는 Reference로 지정
    model.load_adapter(script_args.adapter_name, adapter_name = "reference")
    model.set_adapter("policy")

    training_args.model_adapter_name = "policy"
    training_args.ref_adapter_name = "reference"

    if training_args.gradient_checkpointing:
        model.gradient_checkpointing_enable()

    trainer = DPOTrainer(
        model,
        args = training_args,
        train_dataset= train_ds,
        eval_dataset = test_ds,
        processing_class = tokenizer
    )

    checkpoint = None
    if training_args.resume_from_checkpoint is not None:
        checkpoint = training_args.resume_from_checkpoint

    trainer.train(resume_from_checkpoint = checkpoint)
    trainer.save_model()

if __name__ == "__main__":
    
    parser = TrlParser((ScriptArguments, SFTConfig, LoraArguments))         ## 따로 저장된 파라미터 파싱
    script_args, training_args, lora_args = parser.parse_args_and_config(
        args=["--config", config_path]
    )

    ## Lora Config에 유효한 입력값만 받을 수 있도록 커스터마이징. 원래 TrlParser에는 LoraConfig를 넣지 못함
    valid_keys = LoraConfig.__init__.__code__.co_varnames
    lora_kwargs = {
        f.name: getattr(lora_args, f.name)
        for f in fields(lora_args)
        if f.name in valid_keys
    }

    # seeding(training_args.seed)

    main(script_args, training_args, lora_kwargs)

    print("========== 학습 종료 ==========")

if __name__ == "__main__":
    config_path = "./main/config/DPO_config.yaml"
    parser = TrlParser((ScriptArguments, DPOConfig))
    script_args, training_args, lora_args = parser.parse_args_and_config(
        args=["--config", config_path]
    )

    main(script_args, training_args)

    print("========== 학습 종료 ==========")ㅁ

    ## ========== wandb 업로드 ==========
    os.system(f"wandb sync --include-offline wandb/latest-run") ## 가장 마지막에 진행한 튜닝 작업의 로그 제출

## **7. (2차) 최종 튜닝 모델 테스트**

* HuggingFace 라이브러리를 통해 한 개의 텍스트에 대한 생성을 비교 (원본 / SFT / SFT + DPO)

`-` VRAM 초기화

In [ ]:
gc.collect()
torch.cuda.empty_cache()

`-` 테스트에 활용될 데이터 선택

In [ ]:
## 무작위 1개 텍스트 추출
test_dataset = load_dataset("json", data_files = os.path.join("", "data/sft_test_dataset.json"), split = "train")
random_idx = random.randint(0, len(test_dataset))

## 출력하여 확인
for k, v in test_dataset[random_idx].items():
    print(f"{k}: {v[0]["content"]}\n\n")

messages = test_dataset[random_idx]["prompt"]
chosen = test_dataset[random_idx]["chosen"]

`-` 원시 모델 결과 확인

In [ ]:
## 모델 로드 및 초기 설정
origin_model_name = "meta-llama/Llama-3.1-8B-Instruct"

origin_model = AutoModelForCausalLM.from_pretrained(origin_model_name, use_cache = False, device_map = "cuda:0", dtype = torch.bfloat16)
origin_tokenizer = AutoTokenizer.from_pretrained(origin_model_name, use_fast = True)
origin_tokenizer.pad_token = origin_tokenizer.eos_token
origin_tokenizer.pad_token_id = origin_tokenizer.eos_token_id
origin_tokenizer.padding_side = "left"

terminators = [origin_tokenizer.eos_token_id]

input_ids = origin_tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,   ## 생성 시에 맨 뒤 generation 시작하라는 프롬프트 삽입
    return_tensors = "pt").to(origin_model.device)

## 생성
outputs = origin_model.generate(
    input_ids,
    max_new_tokens = 512,
    eos_token_id = terminators,
    do_sample = True,
    temperature = 0.7,
    top_p = 0.95
)

response = outputs[0][input_ids.shape[-1]:]
print(f"prompt:\n{messages[0]["content"]}\n")
print(f"chosen:\n{chosen[0]["content"]}\n")
print(f"generate:\n{origin_tokenizer.decode(response, skip_special_tokens = True)}")

`-` 베이스 모델과 SFT 어뎁터 로드 및 비교

* 베이스 모델을 4비트로 양자화한 뒤, SFT 어뎁터를 부착

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.bfloat16
)

adapter_name = "../adapter/SFT_Adapter" ## 마지막 체크포인트 로드 (상황에 따라 조정)

## 양자화 모델 로드
sft_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.1-8B-Instruct",
    quantization_config = bnb_config,
    attn_implementation = "eager",
    use_cache = True,
    dtype = torch.bfloat16,
    device_map = "cuda:0"
)

sft_model = PeftModel.from_pretrained(sft_model, adapter_name)  ## 어뎁터 부착
tokenizer = AutoTokenizer.from_pretrained(adapter_name, use_fast = True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"

terminators = [tokenizer.eos_token_id]

## 생성
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,   ## 생성 시에 맨 뒤 generation 시작하라는 프롬프트 삽입
    return_tensors = "pt").to(sft_model.device)

outputs = sft_model.generate(
    input_ids,
    max_new_tokens = 512,       ## prompt 제외 출력 토큰 수. max_length는 prompt 포함
    eos_token_id = terminators,
    do_sample = True,
    temperature = 0.7,
    top_p = 0.95
)

response = outputs[0][input_ids.shape[-1]:]
print(f"prompt:\n{messages[0]["content"]}\n")
print(f"chosen:\n{chosen[0]["content"]}\n")
print(f"generate:\n{tokenizer.decode(response, skip_special_tokens = True)}")

In [ ]:
del sft_model
del tokenizer
gc.collect()
torch.cuda.empty_cache()

`-` 베이스 모델과 DPO 어뎁터 로드 및 비교

* 베이스 모델을 4비트로 양자화한 뒤, DPO 어뎁터를 부착

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.bfloat16
)

adapter_name = "../adapter/DPO_Adapter" ## 마지막 체크포인트 로드 (상황에 따라 조정)

## 양자화 모델 로드
dpo_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.1-8B-Instruct",
    quantization_config = bnb_config,
    attn_implementation = "eager",
    use_cache = True,
    dtype = torch.bfloat16,
    device_map = "cuda:0"
)

dpo_model = PeftModel.from_pretrained(dpo_model, adapter_name)  ## 어뎁터 부착
tokenizer = AutoTokenizer.from_pretrained(adapter_name, use_fast = True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"

terminators = [tokenizer.eos_token_id]

## 생성
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,   ## 생성 시에 맨 뒤 generation 시작하라는 프롬프트 삽입
    return_tensors = "pt").to(dpo_model.device)

outputs = dpo_model.generate(
    input_ids,
    max_new_tokens = 512,       ## prompt 제외 출력 토큰 수. max_length는 prompt 포함
    eos_token_id = terminators,
    do_sample = True,
    temperature = 0.7,
    top_p = 0.95
)

response = outputs[0][input_ids.shape[-1]:]
print(f"prompt:\n{messages[0]["content"]}\n")
print(f"chosen:\n{chosen[0]["content"]}\n")
print(f"generate:\n{tokenizer.decode(response, skip_special_tokens = True)}")

In [ ]:
del dpo_model
del tokenizer
gc.collect()
torch.cuda.empty_cache()

## **8. vLLM Inference**

* 모델의 빌드 및 최종 추론은 vLLM으로 수행
* 대규모 데이터셋에서 HuggingFace 라이브러리보다 무조건 훨씬 빠름